### PLAN
1. Load up to Jun 2022 into BigQuery
2. Create github action to run BQ insert script nightly
3. Set grouped database table (history_df) in BigQuery to refresh nightly after insert
4. Make script to pull data history_df from BigQuery data

8. For streamlit app append [history_df, current_df, future_df] to make the combo_df
    - put history_df into cache using st.experimental_memo so its only called once each time app is opened

In [14]:
# Initialize python libs & SQL creads
from datetime import datetime, timedelta, date
import pandas as pd
from pandasql import sqldf
import streamlit as st
import pull_nrg_data
import json
import http.client
import certifi
import ssl
import os
import timeit
import pickle

In [ ]:
pull_nrg_data.release_token(accessToken)

In [12]:
# Release token
import certifi
import ssl
import http.client
server = 'api.nrgstream.com'

def release_token(accessToken):
    path = '/api/ReleaseToken'
    headers = {'Authorization': f'Bearer {accessToken}'}
    context = ssl.create_default_context(cafile=certifi.where())
    conn = http.client.HTTPSConnection(server,context=context)
    conn.request('DELETE', path, None, headers)
    res = conn.getresponse()
    print('token released')

accessToken = 'GPoLkjwW2Y1qxiWP6IUvNGnumicrW__QZBIBpfdkyb2a532BeAQwnVkYyASTVqOLozWYD8_RCcNUKLV2mVUxvl0sPwnDstyqkY4xyn_OkatfPfOhloZQLe6alUXSUyPyoMhS68u_MpEHsHdoSyhyIJHh_0UTfp9M0GC6Z70ds4yxdXXeL5bTGyKafOS8SXs67Y3E6fSUS5kuVz36YqIb92fzyMwtqHZeSGud2ExIhB36ax5yrSvbJWkWF2TQhkNNbRgWU5TdJrwe83opC5YaGiguIJmQP9JyGiZ_luz4F30TzkruqyAVS044OGYFya8lYLBAwk5Jwtd7JAK_Tgt6ByzyXfQ'
release_token(accessToken)

token released


In [99]:
# Joining daily to hourly data
import pandas as pd

intraday = pd.DataFrame(index=pd.date_range('2016-01-01', '2016-01-07', freq='H'),data=[i for i in range(145)], columns=['hourly'])
daily = pd.DataFrame(index=pd.date_range('2016-01-01', '2016-01-07', freq='D'), data=[i for i in range(7)], columns=['daily'])
df = intraday.join(daily).fillna(method='ffill')
intraday, daily, df

(                     hourly
 2016-01-01 00:00:00       0
 2016-01-01 01:00:00       1
 2016-01-01 02:00:00       2
 2016-01-01 03:00:00       3
 2016-01-01 04:00:00       4
 ...                     ...
 2016-01-06 20:00:00     140
 2016-01-06 21:00:00     141
 2016-01-06 22:00:00     142
 2016-01-06 23:00:00     143
 2016-01-07 00:00:00     144
 
 [145 rows x 1 columns],
             daily
 2016-01-01      0
 2016-01-02      1
 2016-01-03      2
 2016-01-04      3
 2016-01-05      4
 2016-01-06      5
 2016-01-07      6,
                      hourly  daily
 2016-01-01 00:00:00       0    0.0
 2016-01-01 01:00:00       1    0.0
 2016-01-01 02:00:00       2    0.0
 2016-01-01 03:00:00       3    0.0
 2016-01-01 04:00:00       4    0.0
 ...                     ...    ...
 2016-01-06 20:00:00     140    5.0
 2016-01-06 21:00:00     141    5.0
 2016-01-06 22:00:00     142    5.0
 2016-01-06 23:00:00     143    5.0
 2016-01-07 00:00:00     144    6.0
 
 [145 rows x 2 columns])

In [83]:
with open('./intertie.pickle', 'rb') as handle:
        intertie = pickle.load(handle)
intertie

,timeStamp,value,fuelType
0,2022-01-01 00:00:00,750,Intertie
1,2022-01-01 01:00:00,750,Intertie
2,2022-01-01 02:00:00,750,Intertie
3,2022-01-01 03:00:00,750,Intertie
4,2022-01-01 04:00:00,750,Intertie
...,...,...,...
5587,2023-08-21 19:00:00,750,Intertie
5588,2023-08-21 20:00:00,750,Intertie
5589,2023-08-21 21:00:00,750,Intertie
5590,2023-08-21 22:00:00,750,Intertie


In [38]:
with open('./alerts.pickle', 'wb') as handle:
    pickle.dump(alerts, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [117]:
mx = max(intertie['timeStamp'])
mn = min(intertie['timeStamp'])
date_index = pd.date_range(mn, mx, freq='H')
daily = pd.DataFrame(index=date_index, data=[i for i in range(len(date_index))], columns=['daily'])
daily.join(outages.set_index('timeStamp')).fillna(method='ffill').head(50)

,daily,value,fuelType
2022-01-01 00:00:00,0,700.0,Coal
2022-01-01 00:00:00,0,740.0,Natural Gas
2022-01-01 00:00:00,0,0.0,Dual Fuel
2022-01-01 00:00:00,0,120.0,Hydro
2022-01-01 00:00:00,0,130.0,Wind
2022-01-01 00:00:00,0,400.0,Solar
2022-01-01 00:00:00,0,0.0,Energy Storage
2022-01-01 00:00:00,0,50.0,Biomass & Other
2022-01-01 01:00:00,1,50.0,Biomass & Other
2022-01-01 02:00:00,2,50.0,Biomass & Other


In [112]:
pd.date_range(mn, mx, freq='H')

DatetimeIndex(['2022-01-01 00:00:00', '2022-01-01 01:00:00',
               '2022-01-01 02:00:00', '2022-01-01 03:00:00',
               '2022-01-01 04:00:00', '2022-01-01 05:00:00',
               '2022-01-01 06:00:00', '2022-01-01 07:00:00',
               '2022-01-01 08:00:00', '2022-01-01 09:00:00',
               ...
               '2023-08-21 14:00:00', '2023-08-21 15:00:00',
               '2023-08-21 16:00:00', '2023-08-21 17:00:00',
               '2023-08-21 18:00:00', '2023-08-21 19:00:00',
               '2023-08-21 20:00:00', '2023-08-21 21:00:00',
               '2023-08-21 22:00:00', '2023-08-21 23:00:00'],
              dtype='datetime64[ns]', length=14352, freq='H')

In [95]:
outages

,timeStamp,value,fuelType
0,2022-01-01,700,Coal
1,2022-02-01,0,Coal
2,2022-03-01,80,Coal
3,2022-04-01,310,Coal
4,2022-05-01,0,Coal
...,...,...,...
3,2024-04-01,30,Biomass & Other
4,2024-05-01,30,Biomass & Other
5,2024-06-01,30,Biomass & Other
6,2024-07-01,30,Biomass & Other


In [96]:
intertie

,timeStamp,value,fuelType
0,2022-01-01 00:00:00,750,Intertie
1,2022-01-01 01:00:00,750,Intertie
2,2022-01-01 02:00:00,750,Intertie
3,2022-01-01 03:00:00,750,Intertie
4,2022-01-01 04:00:00,750,Intertie
...,...,...,...
5587,2023-08-21 19:00:00,750,Intertie
5588,2023-08-21 20:00:00,750,Intertie
5589,2023-08-21 21:00:00,750,Intertie
5590,2023-08-21 22:00:00,750,Intertie
